# March Madness Simulator

In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import class_functions as cf

In [2]:
class MarchMadnessSimulator:
    def __init__(self, year):
        self.year = year
        self.tourney_seeds = pd.read_csv('resources/DataFiles/NCAATourneySeeds.csv')
        self.teams = self.tourney_seeds[['Season', 'Seed', 'TeamID']]
        #W
        self.east_prelim = {'pair11': [], 'pair16': []}
        self.east = {'pair1': [],'pair2': [],'pair3': [],'pair4': [],'pair5': [],'pair6': [],'pair7': [],'pair8': []}
        #X
        self.midwest_prelim = {'pair11': [], 'pair16': []}
        self.midwest = {'pair1': [],'pair2': [],'pair3': [],'pair4': [],'pair5': [],'pair6': [],'pair7': [],'pair8': []}
        #Y
        self.south_prelim = {'pair11': [], 'pair16': []}
        self.south = {'pair1': [],'pair2': [],'pair3': [],'pair4': [],'pair5': [],'pair6': [],'pair7': [],'pair8': []}
        #Z
        self.west_prelim = {'pair11': [], 'pair16': []}
        self.west = {'pair1': [],'pair2': [],'pair3': [],'pair4': [],'pair5': [],'pair6': [],'pair7': [],'pair8': []}

        self.round_32_matches = {1: 'r1', 2: 'r2', 3: 'r3',4: 'r4',5: 'r4',6: 'r3',7: 'r2',8: 'r1'}
        self.sweet_16_matches = {1: 'p1', 2: 'p2', 3: 'p2', 4: 'p1'}
        self.elite_8_matches = {'east': 'ff1', 'midwest': 'ff1', 'south': 'ff2', 'west': 'ff2'}
        self.matches = {'round_64': self.round_32_matches, 'round_32': self.sweet_16_matches, 'elite_8': self.elite_8_matches}
    
        self.winners = {'first_four': [], 'round_64': [], 'round_32': [], 'sweet_16': [], 
                        'elite_8': [], 'final_4': [], 'championship': [], 'champion': []}
        self.matchups_list = []
        self.preliminaries = {}
        self.preliminary_matchups = []
    def build_bracket(self):
        # setup the first round (round_64, including the preliminaries)
        for t in self.teams.values:
            if t[0] == self.year:
                if t[1][0] == 'W':
                    seed = t[1][1:]
                    if 'a' in seed or 'b' in seed:
                        num = t[1][1:3]
                        self.east_prelim[('pair' + num)].append(cf.getTeamName(t[2]))
                    else:
                        num = int(t[1][1:])
                        self.east[('pair'+str(cf.getPair(num)))].append(cf.getTeamName(t[2]))
                if t[1][0] == 'X':
                    seed = t[1][1:]
                    if 'a' in seed or 'b' in seed:
                        num = t[1][1:3]
                        self.midwest_prelim[('pair' + num)].append(cf.getTeamName(t[2]))
                    else:
                        num = int(t[1][1:])
                        self.midwest[('pair'+str(cf.getPair(num)))].append(cf.getTeamName(t[2]))
                if t[1][0] == 'Y':
                    seed = t[1][1:]
                    if 'a' in seed or 'b' in seed:
                        num = t[1][1:3]
                        self.south_prelim[('pair' + num)].append(cf.getTeamName(t[2]))
                    else:
                        num = int(t[1][1:])
                        self.south[('pair'+str(cf.getPair(num)))].append(cf.getTeamName(t[2]))
                if t[1][0] == 'Z':
                    seed = t[1][1:]
                    if 'a' in seed or 'b' in seed:
                        num = t[1][1:3]
                        self.west_prelim[('pair' + num)].append(cf.getTeamName(t[2]))
                    else:
                        num = int(t[1][1:])
                        self.west[('pair'+str(cf.getPair(num)))].append(cf.getTeamName(t[2]))
    def runPreliminaries(self):
        # run through the preliminaries (First Four)
        self.preliminaries = {'east_prelim': self.east_prelim, 'midwest_prelim': self.midwest_prelim, 
                    'south_prelim': self.south_prelim, 'west_prelim': self.west_prelim}
        for name in self.preliminaries:
            pre_round = self.preliminaries[name]
            count = 0
            for match in pre_round.values():
                prelim = list(pre_round.keys())[count]
                pair = int(prelim.replace('pair', ''))
                if len(match) > 0:
                    # Predict this with classification
                    self.preliminary_matchups.append(match)
                    winner = self.findWinner(match)
                    self.addPrelimWinners(name.replace('_prelim',''), winner, pair)
                count += 1
    def addPrelimWinners(self, region, team, pair):
        if region == 'east':
            self.east[('pair'+str(cf.getPair(pair)))].append(team)
        if region == 'midwest':
            self.midwest[('pair'+str(cf.getPair(pair)))].append(team)
        if region == 'south':
            self.south[('pair'+str(cf.getPair(pair)))].append(team)
        if region == 'west':
            self.west[('pair'+str(cf.getPair(pair)))].append(team)
    def findWinner(self, matchup):
        team1 = matchup[0]
        team2 = matchup[1]
        # first is winner, second is loser
        self.matchups_list.append([team1, team2])
        return team1
    def simulateElite8(self, elite_8):
        final_4 = {}
        for region in elite_8:
            for key in elite_8[region]:
                matchup = elite_8[region][key]
                winner = self.findWinner(matchup)
            final_4[region] = winner
        return final_4
    def simulateFinalFour(self, final_4):
        championship = []
        championship.append(self.findWinner([final_4['east'], final_4['midwest']]))
        championship.append(self.findWinner([final_4['south'], final_4['west']]))
        return championship
    def simulateChampionship(self, championship):
        return self.findWinner(championship)
    def simulateRound(self, prev_round, round_name, matchup_dic):
        next_round = {}
        for region in prev_round:
            bracket_spot = prev_round[region]
            new_round = deepcopy(matchup_dic)       
            for matchup in bracket_spot:            
                m_num = int(matchup[-1])            
                winner = self.findWinner(bracket_spot[matchup])
                if 'elite' in round_name:
                    new_round_spot = self.matches[round_name][region]
                else:
                    new_round_spot = self.matches[round_name][m_num] 
                new_round[new_round_spot].append(winner)
            next_round[region] = new_round
        if 'elite' in round_name:
            del next_round['east']['ff2']
            del next_round['midwest']['ff2']
            del next_round['south']['ff1']
            del next_round['west']['ff1']
        return next_round
    def run_simulation(self):
        print('First Four: ', self.preliminary_matchups, '\n')
        
        self.round_64 = {'east': self.east, 'midwest': self.midwest, 'south': self.south, 'west': self.west}
        print('Round 64: ', self.round_64, '\n')

        self.round_32 = self.simulateRound(self.round_64, 'round_64', {'r1': [], 'r2': [], 'r3': [], 'r4': []})
        print('Round 32: ', self.round_32, '\n')

        self.sweet_16 = self.simulateRound(self.round_32, 'round_32', {'p1': [], 'p2': []})
        print('Sweet 16: ', self.sweet_16, '\n')

        self.elite_8 = self.simulateRound(self.sweet_16, 'elite_8', {'ff1': [], 'ff2': []})
        print('Elite 8: ', self.elite_8, '\n')

        self.final_4 = self.simulateElite8(self.elite_8)
        print('Final 4: ', self.final_4, '\n')
    
        self.championship = self.simulateFinalFour(self.final_4)
        print('Championship: ', self.championship, '\n')

        self.champion = self.simulateChampionship(self.championship)
        print('CHAMPION: ', self.champion, '\n')
    def printBracket(self):
        print('East: ', self.east, '\n')
        print('Midwest: ', self.midwest, '\n')
        print('South: ', self.south, '\n')
        print('West: ', self.west, '\n')

In [3]:
march_madness = MarchMadnessSimulator(2017)

march_madness.build_bracket()

march_madness.runPreliminaries()

march_madness.run_simulation()

#march_madness.matchups_list

First Four:  [['Providence', 'USC'], ["Mt St Mary's", 'New Orleans'], ['NC Central', 'UC Davis'], ['Kansas St', 'Wake Forest']] 

Round 64:  {'east': {'pair1': ['Villanova', "Mt St Mary's"], 'pair2': ['Duke', 'Troy'], 'pair3': ['Baylor', 'New Mexico St'], 'pair4': ['Florida', 'ETSU'], 'pair5': ['Virginia', 'UNC Wilmington'], 'pair6': ['SMU', 'Providence'], 'pair7': ['South Carolina', 'Marquette'], 'pair8': ['Wisconsin', 'Virginia Tech']}, 'midwest': {'pair1': ['Gonzaga', 'S Dakota St'], 'pair2': ['Arizona', 'North Dakota'], 'pair3': ['Florida St', 'FL Gulf Coast'], 'pair4': ['West Virginia', 'Bucknell'], 'pair5': ['Notre Dame', 'Princeton'], 'pair6': ['Maryland', 'Xavier'], 'pair7': ["St Mary's CA", 'VA Commonwealth'], 'pair8': ['Northwestern', 'Vanderbilt']}, 'south': {'pair1': ['Kansas', 'NC Central'], 'pair2': ['Louisville', 'Jacksonville St'], 'pair3': ['Oregon', 'Iona'], 'pair4': ['Purdue', 'Vermont'], 'pair5': ['Iowa St', 'Nevada'], 'pair6': ['Creighton', 'Rhode Island'], 'pair7'

In [4]:
march_madness.tourney_seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2213,2018,Z13,1422
2214,2018,Z14,1285
2215,2018,Z15,1252
2216,2018,Z16a,1300


In [6]:
tourney_results = pd.read_csv('resources/DataFiles/NCAATourneyCompactResults.csv')

#tourney_year_results = tourney_results[tourney_results['Season'] == 2018].values

tourney_results

#tourney_year_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2179,2018,146,1242,85,1181,81,N,1
2180,2018,146,1437,71,1403,59,N,0
2181,2018,152,1276,69,1260,57,N,0
2182,2018,152,1437,95,1242,79,N,0


In [7]:
real_matchups = []
for row in tourney_year_results:
    if row[2] > row[4]:
        real_matchups.append([cf.getTeamName(row[2]), cf.getTeamName(row[4])])
    else:
        real_matchups.append([cf.getTeamName(row[4]), cf.getTeamName(row[2])])
real_matchups

[['Radford', 'Long Island'],
 ['UCLA', 'St Bonaventure'],
 ['Syracuse', 'Arizona St'],
 ['TX Southern', 'NC Central'],
 ['Virginia Tech', 'Alabama'],
 ['Buffalo', 'Arizona'],
 ['Iona', 'Duke'],
 ['St Bonaventure', 'Florida'],
 ['UNC Greensboro', 'Gonzaga'],
 ['San Diego St', 'Houston'],
 ['Penn', 'Kansas'],
 ['Kentucky', 'Davidson'],
 ['Miami FL', 'Loyola-Chicago'],
 ['Montana', 'Michigan'],
 ['S Dakota St', 'Ohio St'],
 ['Rhode Island', 'Oklahoma'],
 ['Seton Hall', 'NC State'],
 ['Wright St', 'Tennessee'],
 ['Texas Tech', 'SF Austin'],
 ['Villanova', 'Radford'],
 ['Col Charleston', 'Auburn'],
 ['Butler', 'Arkansas'],
 ['Georgia St', 'Cincinnati'],
 ['New Mexico St', 'Clemson'],
 ['Missouri', 'Florida St'],
 ['Kansas St', 'Creighton'],
 ['Wichita St', 'Marshall'],
 ['Michigan St', 'Bucknell'],
 ['Texas', 'Nevada'],
 ['North Carolina', 'Lipscomb'],
 ['Purdue', 'CS Fullerton'],
 ['TCU', 'Syracuse'],
 ['Texas A&M', 'Providence'],
 ['Virginia', 'UMBC'],
 ['West Virginia', 'Murray St'],
 ['